- fhs_flashsale_product: (Thông tin flashsale của product)
    * period_id : id của flashsale
    * product_id : id của product 
    * original_price : giá gốc
    * flashsale_price : giá sale
    * total_items : số lượng sale
    
- fhs_flashsale_period: (Chứa thông tin flashsale về thời gian)
    * id: id của flashsale (== period_id của fhs_flashsale_product)
    * start_date, end_date: time bắt đầu kết thúc sale 
    * các cột khác không quan tâm
    
- fhs_catalog_product_entity: (Thông tin discount và ngày bắt đầu bán)
    * entity_id: id của product (==product_id trong fhs_flashsale_product)
    * sku : là mã sản phẩm (duy nhất và là khóa của sản phẩm dùng để truy các thông tin khác về sản phẩm)
    * created_at : ngày bắt đầu bán
    * discount_percent : phần trăm giảm giá so với giá bìa (trừ flashsale còn giá sản phẩm mọi lúc sẽ là giá gốc trừ đi disount này)

- fhs_cataloginventory_stock_item: Bãng này là số lượng sản phẩm tồn nhưng ko dùng được, do bên db bị mất dữ liệu
    
- fhs_sales_flat_order: Chứa thông tin đơn hàng
    * entity_id : id của đơn hàng (==order_id trong fhs_sales_flat_order_item)
    * state : trạng thái đơn hàng
    * status : trạng thái của trạng thái đơn hàng (ví dụ đơn hàng ở state đang giao, status là complete nghĩa là ở trang thái đang * giao đã hoàn thành)
    * created_at : ngày tạo đơn
    * updated_at : ngày cập nhật trạng thái
    * store_id : id của kho, id < 4 cho thương mại điện tử  (nhớ thêm điều kiện store_id<4 khi query bảng này)

- fhs_sales_flat_order_item: Chưa thông tin từng sản phẩm trong đơn hàng
    * order_id: id của đơn hàng (==entity_id trong fhs_sales_flat_order)
    * created_at: Ngày tạo đơn con
    * product_id, sku : id và mã sản phẩm

- fhs_catalog_product_entity_decimal: Bảng giá sản phẩm
    * entity_id: id của product (==entity_id trong fhs_catalog_product_entity)
    * attribute_id : loại giá (giá gốc 75, giá bán 76 (hoặc ngược lại chị không nhớ, chỉ quan tâm 2 loại này))
    * value : giá bán của sản phẩm với attribute tương ứng
    
- fhs_product_stock_log: Bảng này là thông tin các sản phẩm đang bán hiện tại trên thị trường
    * product_id, sku : id và mã của sp
    * qty: số lượng trong kho
    * is_in_stock: 1 là còn bán
    * created_at: không rõ
    * bảng này cũng không đáng tin lắm 
    
- product_dim : Thông tin sp
    * sku: mã
    * name: tên
    * cat1 tới cat5: là categories của sp
    * price : giá gốc

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

- Đọc các file csv vào các biến tương ứng

In [2]:
fhs_flashsale_product = pd.read_csv('../Data/Data_Fahasa/fhs_flashsale_product.csv')
fhs_flashsale_period = pd.read_csv('../Data/Data_Fahasa/fhs_flashsale_period.csv')
fhs_catalog_product_entity = pd.read_csv('../Data/Data_Fahasa/fhs_catalog_product_entity.csv')
fhs_sales_flat_order = pd.read_csv('../Data/Data_Fahasa/fhs_sales_flat_order.csv')
fhs_sales_flat_order_item = pd.read_csv('../Data/Data_Fahasa/fhs_sales_flat_order_item.csv')
fhs_catalog_product_entity_decimal = pd.read_csv('../Data/Data_Fahasa/fhs_catalog_product_entity_decimal.csv')
fhs_product_stock_log = pd.read_csv('../Data/Data_Fahasa/fhs_product_stock_log.csv')
product_dim = pd.read_csv('../Data/Data_Fahasa/product_dim.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Data_Fahasa/fhs_flashsale_product.csv'

- Đổi tên các cột theo tên của từng file cho dễ nhận dạng

In [ ]:
def rename_columns(dataframe, dataframe_name):
    """
    rename columns with patern: dataframe_name.columns_name
    Input:
        dataframe: Dataframe Pandas
        dataframe_name: string
    """
    columns = dataframe.columns
    # rename columns
    new_names = []
    for index in range(len(columns)):
        new_name = f'{dataframe_name}.{columns[index]}'
        new_names.append(new_name)
    
    dataframe.columns = new_names

rename_columns(fhs_flashsale_product, 'fhs_flashsale_product')
rename_columns(fhs_flashsale_period, 'fhs_flashsale_period')
rename_columns(fhs_catalog_product_entity, 'fhs_catalog_product_entity')
rename_columns(fhs_sales_flat_order, 'fhs_sales_flat_order')
rename_columns(fhs_sales_flat_order_item, 'fhs_sales_flat_order_item')
rename_columns(fhs_catalog_product_entity_decimal, 'fhs_catalog_product_entity_decimal')
rename_columns(fhs_product_stock_log, 'fhs_product_stock_log')
rename_columns(product_dim, 'product_dim')

- Xóa các cột không cần thiết của fhs_flashsale_period

In [ ]:
temp = fhs_flashsale_period.loc[:,
    ['fhs_flashsale_period.id', 'fhs_flashsale_period.start_date', 'fhs_flashsale_period.end_date']]
# xóa đi để tiết kiệm bộ nhớ
del fhs_flashsale_period
fhs_flashsale_period = temp

- Chuyển các chuỗi thời gian về định dạng datetime

In [ ]:
def string_to_datetime(dataframe, column_name):
          dataframe[f'{column_name}'] \
          = pd.to_datetime(dataframe[f'{column_name}'],
                              format = '%Y/%m/%d %H:%M:%S')

string_to_datetime(fhs_flashsale_period, 'fhs_flashsale_period.start_date')
string_to_datetime(fhs_flashsale_period, 'fhs_flashsale_period.end_date')
string_to_datetime(fhs_catalog_product_entity, 'fhs_catalog_product_entity.created_at')
string_to_datetime(fhs_sales_flat_order, 'fhs_sales_flat_order.created_at')
string_to_datetime(fhs_sales_flat_order, 'fhs_sales_flat_order.updated_at')
string_to_datetime(fhs_sales_flat_order_item, 'fhs_sales_flat_order_item.created_at')
string_to_datetime(fhs_product_stock_log, 'fhs_product_stock_log.created_at')

In [ ]:
print('fhs_flashsale_product', len(fhs_flashsale_product))
print('fhs_flashsale_period', len(fhs_flashsale_period))
print('fhs_product_stock_log',len(fhs_product_stock_log))
print('fhs_catalog_product_entity',len(fhs_catalog_product_entity))
print('fhs_catalog_product_entity_decimal',len(fhs_catalog_product_entity_decimal))
print('fhs_sales_flat_order',len(fhs_sales_flat_order))
print('fhs_sales_flat_order_item',len(fhs_sales_flat_order_item))
print('product_dim',len(product_dim))

fhs_flashsale_product 3930772
fhs_flashsale_period 6416
fhs_product_stock_log 10251317
fhs_catalog_product_entity 352880
fhs_catalog_product_entity_decimal 1213003
fhs_sales_flat_order 3541823
fhs_sales_flat_order_item 17828291
product_dim 2961504


- Merge thông tin của các sản phẩm flashsale lại với nhau để lấy các thông tin cần thiết như: ()
  * Mã sku
  * Thời gian flashsale của từng sản phẩm
  * ......

In [ ]:
temp = pd.merge(fhs_flashsale_product, fhs_flashsale_period, 
          left_on='fhs_flashsale_product.period_id', right_on='fhs_flashsale_period.id')
fhs_flashsale_product_info = pd.merge(temp, fhs_catalog_product_entity, 
          left_on='fhs_flashsale_product.product_id', right_on='fhs_catalog_product_entity.entity_id')
del temp
fhs_flashsale_product_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3930772 entries, 0 to 3930771
Data columns (total 12 columns):
 #   Column                                       Dtype         
---  ------                                       -----         
 0   fhs_flashsale_product.period_id              int64         
 1   fhs_flashsale_product.product_id             int64         
 2   fhs_flashsale_product.original_price         float64       
 3   fhs_flashsale_product.flashsale_price        float64       
 4   fhs_flashsale_product.total_items            float64       
 5   fhs_flashsale_period.id                      int64         
 6   fhs_flashsale_period.start_date              datetime64[ns]
 7   fhs_flashsale_period.end_date                datetime64[ns]
 8   fhs_catalog_product_entity.entity_id         int64         
 9   fhs_catalog_product_entity.sku               object        
 10  fhs_catalog_product_entity.created_at        datetime64[ns]
 11  fhs_catalog_product_entity.discount_p

- Tương tự với đơn hàng, ta sẽ merge giữa đơn hàng chính và đơn hàng con để tập trung vào các thuộc tính:
  * Trạng thái đơn hàng chính
  * Ngày tạo đơn hàng con

In [ ]:
fhs_sales_flat_order_item_state  \
        = pd.merge(fhs_sales_flat_order, fhs_sales_flat_order_item,
                        left_on = 'fhs_sales_flat_order.entity_id', 
                        right_on = 'fhs_sales_flat_order_item.order_id')

fhs_sales_flat_order_item_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13969403 entries, 0 to 13969402
Data columns (total 12 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   fhs_sales_flat_order.entity_id        int64         
 1   fhs_sales_flat_order.increment_id     float64       
 2   fhs_sales_flat_order.state            object        
 3   fhs_sales_flat_order.status           object        
 4   fhs_sales_flat_order.created_at       datetime64[ns]
 5   fhs_sales_flat_order.updated_at       datetime64[ns]
 6   fhs_sales_flat_order.store_id         int64         
 7   fhs_sales_flat_order_item.item_id     int64         
 8   fhs_sales_flat_order_item.order_id    int64         
 9   fhs_sales_flat_order_item.created_at  datetime64[ns]
 10  fhs_sales_flat_order_item.product_id  float64       
 11  fhs_sales_flat_order_item.sku         object        
dtypes: datetime64[ns](3), float64(2), int64(4), object(3)
memory usage: 

- Vì số lượng dòng quá lớn nên ta sẽ thực hiện chiến lược tách các sản phẩm theo từng nhóm:
  * Theo năm: và mỗi năm chỉ lấy ra những sản phẩm bán chạy nhất hoặc lấy những sản phẩm trung bình
  * Có thể nghĩ đến chiến thuật lấy ngẫu nhiên sau đó dùng thống kê mô tả để dự đoán ảnh hưởng của flashsale đến lưu lượng bán hàng

In [ ]:
# Giả sử xét trong năm 2022
import datetime 
min_year = pd.to_datetime('2022/1/1',
                              format = '%Y/%m/%d %H:%M:%S')
max_year = pd.to_datetime('2022/12/31 23:59:59',
                              format = '%Y/%m/%d %H:%M:%S')

condition2 = np.where((fhs_sales_flat_order_item_state['fhs_sales_flat_order_item.created_at'] >=  min_year)
          &           (fhs_sales_flat_order_item_state['fhs_sales_flat_order_item.created_at'] <=  max_year))     
fhs_sales_flat_order_item_state_2022 = fhs_sales_flat_order_item_state.loc[condition2] 

# Tương ứng với những sản phẩm được tung ra trong khoảng thời gian này 2022
condition3 = np.where((fhs_flashsale_product_info['fhs_flashsale_period.start_date'] >=  min_year)
          &           (fhs_flashsale_product_info['fhs_flashsale_period.start_date'] <=  max_year))
fhs_flashsale_product_info_2022 = fhs_flashsale_product_info.loc[condition3] 

In [73]:
fhs_sales_flat_order_item_state_2022 = pd.read_csv('Data/fhs_sales_flat_order_item_state_2022.csv')
fhs_flashsale_product_info_2022 = pd.read_csv('Data/fhs_flashsale_product_info_2022.csv')

C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [74]:
print(len(fhs_sales_flat_order_item_state_2022))
print(len(fhs_flashsale_product_info_2022))

997797
797530


- Tạo thêm 2 cột giá trị, để có sau này truy vấn đơn giản hơn cho fhs_sales_flat_order_item_state:
  * flashsale_product: Có phải là đơn hàng flashsale hay không
  * flashsale_day: True là trong ngày có flashsale, False ngược lại
  * flashsale_time: True là trong khung giờ có flashsale, False ngược lại


- Đầu tiên là xác định có phải đơn hàng flashsale hay không

In [75]:
fhs_sales_flat_order_item_state_2022['flashsale_product'] = fhs_sales_flat_order_item_state_2022['fhs_sales_flat_order_item.sku']\
          .isin(fhs_flashsale_product_info_2022['fhs_catalog_product_entity.sku'])


- Tiếp theo là xem có ở trong ngày flashsale hay không

In [76]:
len(fhs_sales_flat_order_item_state_2022)

997797

In [77]:
fhs_flashsale_product_info_2022.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [78]:
# xác định các đơn hàng được đặt trong khoảng thời gian này
# query các cột có cùng mãu sku
temp = \
          pd.merge(fhs_sales_flat_order_item_state_2022, fhs_flashsale_product_info_2022, how = 'inner',
                    left_on = 'fhs_sales_flat_order_item.sku', right_on='fhs_catalog_product_entity.sku')

# sau đó đặt điều kiện về ngày sao cho ngày đặt của các sản phẩm phải nằm trong thời gian flashsale
temp = temp.loc[
          (temp['fhs_sales_flat_order_item.created_at'] \
                    > temp['fhs_flashsale_period.start_date'])
&         (temp['fhs_sales_flat_order_item.created_at'] \
                    < temp['fhs_flashsale_period.end_date'])]
# temp.drop_duplicates(subset = fhs_sales_flat_order_item_state_2022.columns, 
#           inplace = True)

In [79]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71280 entries, 15545 to 12389209
Data columns (total 26 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   71280 non-null  int64  
 1   fhs_sales_flat_order.entity_id               71280 non-null  int64  
 2   fhs_sales_flat_order.increment_id            71280 non-null  float64
 3   fhs_sales_flat_order.state                   71280 non-null  object 
 4   fhs_sales_flat_order.status                  71280 non-null  object 
 5   fhs_sales_flat_order.created_at              71280 non-null  object 
 6   fhs_sales_flat_order.updated_at              71280 non-null  object 
 7   fhs_sales_flat_order.store_id                71280 non-null  int64  
 8   fhs_sales_flat_order_item.item_id            71280 non-null  int64  
 9   fhs_sales_flat_order_item.order_id           71280 non-null  int6

In [80]:
temp.drop_duplicates(inplace=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70187 entries, 15545 to 12389209
Data columns (total 26 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   70187 non-null  int64  
 1   fhs_sales_flat_order.entity_id               70187 non-null  int64  
 2   fhs_sales_flat_order.increment_id            70187 non-null  float64
 3   fhs_sales_flat_order.state                   70187 non-null  object 
 4   fhs_sales_flat_order.status                  70187 non-null  object 
 5   fhs_sales_flat_order.created_at              70187 non-null  object 
 6   fhs_sales_flat_order.updated_at              70187 non-null  object 
 7   fhs_sales_flat_order.store_id                70187 non-null  int64  
 8   fhs_sales_flat_order_item.item_id            70187 non-null  int64  
 9   fhs_sales_flat_order_item.order_id           70187 non-null  int6

In [84]:
count = 0
for i in temp.duplicated(subset = fhs_sales_flat_order_item_state_2022.columns):
    count+=1
    if i == True:
        print(count)

1595
1600
2556
2558
4740
6455
6457
6462
6722
7000
8024
8869
9039
9041
9043
10026
10028
10030
10278
11093
11095
11097
11099
11101
11554
11644
11646
13265
13840
13842
13844
14053
14055
14307
14309
14311
14992
16282
16398
16437
16499
18462
18464
18508
19361
19363
19365
19367
19369
19371
19604
19606
19608
19610
19612
19614
19616
19618
19620
19622
19624
19626
19628
19630
19632
19634
19636
19638
19640
19642
19644
19876
19959
20035
20318
20320
20322
20324
20326
20740
20742
20744
20746
20748
20750
20752
21783
22577
22731
22873
22875
23376
23378
23410
23509
23650
23652
23654
23656
23658
23660
24292
24460
24489
24611
24631
24633
24635
24637
24639
24641
24970
25152
25154
25156
25233
25256
25267
25269
25285
25397
25871
25873
25875
25877
25879
25916
26231
26931
26933
27017
27019
27021
27023
27134
27506
27508
27530
27650
27909
27911
28636
28663
28666
29029
29393
29395
29397
29399
29475
29477
29833
29835
30582
30868
30870
30872
30874
30876
30878
30880
30882
30884
30886
30888
31958
31961
32088
32286
3

In [92]:
index = 1595 - 1
aa = temp.loc[(temp['fhs_sales_flat_order_item.sku'] == temp.iloc[index]['fhs_sales_flat_order_item.sku'])
       & (temp['fhs_sales_flat_order_item.created_at'] == temp.iloc[index]['fhs_sales_flat_order_item.created_at'])
       & (temp['fhs_sales_flat_order.created_at'] == temp.iloc[index]['fhs_sales_flat_order.created_at'])
       & (temp['fhs_sales_flat_order.updated_at'] == temp.iloc[index]['fhs_sales_flat_order.updated_at'])]
for columns in temp.columns:
       if aa.iloc[0][columns] != aa.iloc[1][columns]:
              print(columns)
              print(aa.iloc[0][columns])
              print(aa.iloc[1][columns])

fhs_flashsale_product.period_id
7944
7961
fhs_flashsale_product.flashsale_price
10000.0
21250.0
fhs_flashsale_product.total_items
10.0
20.0
fhs_flashsale_period.id
7944
7961


- Khác period id trong flash sale nhưng vẫn cùng ngày 
- Có thể lược bỏ, tuy nhiên có thể dẫn đến sai sót khi tính số lượng bán hàng chia cho total item